# Dropout
### 1) import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.0


### 2) Checkpoint Function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### 3) Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

### 4) Performance Function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True) # trainig을 True로 하면 Dropout 사용
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False) # trainig을 False로 하면 Dropout 사용 X
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

### 5) Model Function
- Dropout Function도 정의해준다.

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate):
    return tf.keras.layers.Dropout(rate)

### 6) Create Model(function version)
- dropout 함수 호출(Activation Function 다음)

In [6]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.he_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

### 7) Define data & hyper-parameter

In [7]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

# Graph Input using Dataset API
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

### 8) Define Model & Optimizer & Writer

In [8]:
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### 9) Restore checkpoint & start train or test phase

In [9]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.6440, train_loss: 3.18923926, train_accuracy: 0.1719, test_Accuracy: 0.1505
Epoch: [ 0] [    1/  468] time: 2.7390, train_loss: 3.03414297, train_accuracy: 0.1875, test_Accuracy: 0.1234
Epoch: [ 0] [    2/  468] time: 3.6680, train_loss: 2.81615210, train_accuracy: 0.1328, test_Accuracy: 0.1514
Epoch: [ 0] [    3/  468] time: 4.6050, train_loss: 2.50784397, train_accuracy: 0.2188, test_Accuracy: 0.1930
Epoch: [ 0] [    4/  468] time: 5.5120, train_loss: 2.65920377, train_accuracy: 0.2266, test_Accuracy: 0.2379
Epoch: [ 0] [    5/  468] time: 6.4070, train_loss: 2.69667244, train_accuracy: 0.2812, test_Accuracy: 0.2912
Epoch: [ 0] [    6/  468] time: 7.4370, train_loss: 2.35225606, train_accuracy: 0.4297, test_Accuracy: 0.3411
Epoch: [ 0] [    7/  468] time: 8.4970, train_loss: 2.49049878, train_accuracy: 0.4062, test_Accuracy: 0.3963
Epoch: [ 0] [    8/  468] time: 9.4520,

Epoch: [ 0] [   74/  468] time: 70.6417, train_loss: 1.02241135, train_accuracy: 0.8594, test_Accuracy: 0.8469
Epoch: [ 0] [   75/  468] time: 71.4597, train_loss: 0.88186526, train_accuracy: 0.8203, test_Accuracy: 0.8515
Epoch: [ 0] [   76/  468] time: 72.2727, train_loss: 0.84377497, train_accuracy: 0.8516, test_Accuracy: 0.8545
Epoch: [ 0] [   77/  468] time: 73.0897, train_loss: 0.92919296, train_accuracy: 0.8125, test_Accuracy: 0.8591
Epoch: [ 0] [   78/  468] time: 73.9207, train_loss: 0.90286821, train_accuracy: 0.8594, test_Accuracy: 0.8610
Epoch: [ 0] [   79/  468] time: 74.7467, train_loss: 0.90894639, train_accuracy: 0.8594, test_Accuracy: 0.8628
Epoch: [ 0] [   80/  468] time: 75.6867, train_loss: 0.73570728, train_accuracy: 0.8594, test_Accuracy: 0.8604
Epoch: [ 0] [   81/  468] time: 76.6247, train_loss: 0.87651742, train_accuracy: 0.8125, test_Accuracy: 0.8601
Epoch: [ 0] [   82/  468] time: 77.5847, train_loss: 0.81951773, train_accuracy: 0.8203, test_Accuracy: 0.8600
E

Epoch: [ 0] [  148/  468] time: 135.3471, train_loss: 0.52307898, train_accuracy: 0.8906, test_Accuracy: 0.9096
Epoch: [ 0] [  149/  468] time: 136.2831, train_loss: 0.53675902, train_accuracy: 0.8984, test_Accuracy: 0.9099
Epoch: [ 0] [  150/  468] time: 137.2431, train_loss: 0.42087722, train_accuracy: 0.9297, test_Accuracy: 0.9096
Epoch: [ 0] [  151/  468] time: 138.1621, train_loss: 0.55928344, train_accuracy: 0.9297, test_Accuracy: 0.9097
Epoch: [ 0] [  152/  468] time: 139.0311, train_loss: 0.61113214, train_accuracy: 0.9219, test_Accuracy: 0.9115
Epoch: [ 0] [  153/  468] time: 139.9161, train_loss: 0.62568074, train_accuracy: 0.9062, test_Accuracy: 0.9116
Epoch: [ 0] [  154/  468] time: 140.7541, train_loss: 0.50034332, train_accuracy: 0.9297, test_Accuracy: 0.9120
Epoch: [ 0] [  155/  468] time: 141.6001, train_loss: 0.57918131, train_accuracy: 0.8906, test_Accuracy: 0.9113
Epoch: [ 0] [  156/  468] time: 142.4381, train_loss: 0.49908271, train_accuracy: 0.9375, test_Accuracy:

Epoch: [ 0] [  222/  468] time: 199.0421, train_loss: 0.50251365, train_accuracy: 0.9219, test_Accuracy: 0.9290
Epoch: [ 0] [  223/  468] time: 199.8911, train_loss: 0.45920312, train_accuracy: 0.9141, test_Accuracy: 0.9304
Epoch: [ 0] [  224/  468] time: 200.7021, train_loss: 0.53305244, train_accuracy: 0.8906, test_Accuracy: 0.9305
Epoch: [ 0] [  225/  468] time: 201.5151, train_loss: 0.33523598, train_accuracy: 0.9219, test_Accuracy: 0.9292
Epoch: [ 0] [  226/  468] time: 202.3321, train_loss: 0.47283119, train_accuracy: 0.9219, test_Accuracy: 0.9273
Epoch: [ 0] [  227/  468] time: 203.1711, train_loss: 0.51930511, train_accuracy: 0.9219, test_Accuracy: 0.9260
Epoch: [ 0] [  228/  468] time: 203.9801, train_loss: 0.47927487, train_accuracy: 0.8906, test_Accuracy: 0.9262
Epoch: [ 0] [  229/  468] time: 204.7991, train_loss: 0.50860596, train_accuracy: 0.9375, test_Accuracy: 0.9263
Epoch: [ 0] [  230/  468] time: 205.6091, train_loss: 0.54076958, train_accuracy: 0.8906, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 263.2511, train_loss: 0.27179843, train_accuracy: 0.9609, test_Accuracy: 0.9332
Epoch: [ 0] [  297/  468] time: 264.2181, train_loss: 0.27898121, train_accuracy: 0.9531, test_Accuracy: 0.9332
Epoch: [ 0] [  298/  468] time: 265.3751, train_loss: 0.50782186, train_accuracy: 0.9062, test_Accuracy: 0.9338
Epoch: [ 0] [  299/  468] time: 266.2261, train_loss: 0.34895620, train_accuracy: 0.9219, test_Accuracy: 0.9340
Epoch: [ 0] [  300/  468] time: 267.0391, train_loss: 0.28242722, train_accuracy: 0.9141, test_Accuracy: 0.9345
Epoch: [ 0] [  301/  468] time: 267.8771, train_loss: 0.65038997, train_accuracy: 0.9219, test_Accuracy: 0.9359
Epoch: [ 0] [  302/  468] time: 268.6871, train_loss: 0.46337175, train_accuracy: 0.9219, test_Accuracy: 0.9379
Epoch: [ 0] [  303/  468] time: 269.5171, train_loss: 0.33324778, train_accuracy: 0.9609, test_Accuracy: 0.9384
Epoch: [ 0] [  304/  468] time: 270.3311, train_loss: 0.29094923, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 324.4161, train_loss: 0.13168815, train_accuracy: 0.9688, test_Accuracy: 0.9438
Epoch: [ 0] [  371/  468] time: 325.2441, train_loss: 0.33395165, train_accuracy: 0.9297, test_Accuracy: 0.9440
Epoch: [ 0] [  372/  468] time: 326.0491, train_loss: 0.33762449, train_accuracy: 0.9375, test_Accuracy: 0.9437
Epoch: [ 0] [  373/  468] time: 326.8621, train_loss: 0.40218318, train_accuracy: 0.9531, test_Accuracy: 0.9445
Epoch: [ 0] [  374/  468] time: 327.6721, train_loss: 0.30089957, train_accuracy: 0.9453, test_Accuracy: 0.9435
Epoch: [ 0] [  375/  468] time: 328.5011, train_loss: 0.45706373, train_accuracy: 0.9219, test_Accuracy: 0.9450
Epoch: [ 0] [  376/  468] time: 329.3081, train_loss: 0.40952557, train_accuracy: 0.9375, test_Accuracy: 0.9434
Epoch: [ 0] [  377/  468] time: 330.1291, train_loss: 0.35125011, train_accuracy: 0.9531, test_Accuracy: 0.9434
Epoch: [ 0] [  378/  468] time: 330.9441, train_loss: 0.24145097, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 386.0301, train_loss: 0.24561390, train_accuracy: 0.9766, test_Accuracy: 0.9499
Epoch: [ 0] [  445/  468] time: 386.8421, train_loss: 0.28266257, train_accuracy: 0.9453, test_Accuracy: 0.9491
Epoch: [ 0] [  446/  468] time: 387.6501, train_loss: 0.15478778, train_accuracy: 0.9609, test_Accuracy: 0.9486
Epoch: [ 0] [  447/  468] time: 388.4841, train_loss: 0.46828657, train_accuracy: 0.9375, test_Accuracy: 0.9488
Epoch: [ 0] [  448/  468] time: 389.2951, train_loss: 0.33315670, train_accuracy: 0.9531, test_Accuracy: 0.9490
Epoch: [ 0] [  449/  468] time: 390.1061, train_loss: 0.21338505, train_accuracy: 0.9453, test_Accuracy: 0.9497
Epoch: [ 0] [  450/  468] time: 390.9191, train_loss: 0.27171248, train_accuracy: 0.9688, test_Accuracy: 0.9498
Epoch: [ 0] [  451/  468] time: 391.7411, train_loss: 0.30869722, train_accuracy: 0.9531, test_Accuracy: 0.9501
Epoch: [ 0] [  452/  468] time: 392.5611, train_loss: 0.31188899, train_accuracy: 0.9375, test_Accuracy:

## 3. 결론
> Dropout 기법을 쓰면, 좀 더 높은 accuracy의 모델을 만들 수 있다.

### Using Dropout
- Test Accuracy : 95.09%